In [1]:

import requests, csv
from bs4 import BeautifulSoup

def get_movies(url):
    titles = []
    rankings = []
    years = []
    ratings = []

    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    response = requests.get(url, headers=headers)
    soup_imdb = BeautifulSoup(response.text)

    all_movies = soup_imdb.find_all("h3", {"class":"ipc-title__text"})
    for title in all_movies:
        text = title.getText()

        #check if text if movie title (has number rank)
        if text[0].isnumeric():
            i = text.find(".")
            rankings.append(text[0:i])
            titles.append(text[(i+2):])

    movie_tag = soup_imdb.find_all("span", {"class":"sc-b0691f29-8 ilsLEX cli-title-metadata-item"})
    for tag in range(0, len(movie_tag), 3):
        years.append(movie_tag[tag].getText())

    rating_label = soup_imdb.find_all("span", {"class":"ipc-rating-star ipc-rating-star--base ipc-rating-star--imdb ratingGroup--imdb-rating"})
    for tag in rating_label:
        txt = tag.get("aria-label")
        ratings.append(txt[13:])

    mega_list = list(zip(rankings, titles, years, ratings))
    mega_list = [list(elt) for elt in mega_list]

    # Write csv
    str_lst = [f"{mega_list[i][0]} / {mega_list[i][1]} / {mega_list[i][2]} / {mega_list[i][3]}" for i, val in enumerate(mega_list)]

    res = [elt.split("/") for elt in str_lst]

    with open("imdb_top_250.csv", "w") as f:
        writer = csv.writer(f, delimiter="-")
        writer.writerow(["Ranking", "Title", "Year", "Rating"])
        writer.writerows(res)

In [2]:
def main():
    url = "https://www.imdb.com/chart/top"
    get_movies(url)


if __name__ == "__main__":
    main()

In [3]:
import pandas as pd

df = pd.read_csv("imdb_top_250.csv", encoding='latin-1', delimiter='-')
df.head(5)

,Ranking,Title,Year,Rating
0,1,The Shawshank Redemption,1994,9.3
1,2,The Godfather,1972,9.2
2,3,The Dark Knight,2008,9.0
3,4,The Godfather: Part II,1974,9.0
4,5,12 Angry Men,1957,9.0
